<a href="https://colab.research.google.com/github/Seojooyeon-creat/hakup/blob/main/KLUE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers datasets torch scipy
!pip install git+https://github.com/SKTBrain/KoBERT.git#egg=kobert

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-fk8ruo0l/kobert_e1927caa6bd54f49a0dde678a8386063
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-fk8ruo0l/kobert_e1927caa6bd54f49a0dde678a8386063
  Resolved https://github.com/SKTBrain/KoBERT.git to commit fcd729f2f4b37858f333597c0782388ada51eb5f
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from scipy.stats import spearmanr
from tqdm.auto import tqdm
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 디바이스: {device}")

사용 중인 디바이스: cpu


In [ ]:
model_names = [
    'skt/kobert-base-v1',                     # 1. KoBERT (SKT)
    'klue/bert-base',                         # 2. KLUE-BERT (KLUE 벤치마크 기반)
    'jhgan/ko-sroberta-multitask'             # 3. 한국어 문장 임베딩 특화 모델
]

# 결과를 저장할 딕셔너리
results = {}

In [ ]:
# KLUE STS 데이터셋 로드 (검증용 데이터)
# STS: Semantic Textual Similarity (의미론적 텍스트 유사성)
try:
    sts_dataset = load_dataset('klue', 'sts', split='validation')
except Exception as e:
    print(f"데이터셋 로드 오류: {e}")
    print("인터넷 연결을 확인하거나, 데이터셋 이름을 확인해주세요.")
    # 예외 발생 시, 간단한 예제 데이터로 대체
    sts_dataset = [
        {'sentence1': '고양이가 소파에서 자고 있다.', 'sentence2': '캣이 카우치에서 잠을 잔다.', 'labels': {'label': 5.0}},
        {'sentence1': '오늘 날씨가 매우 맑다.', 'sentence2': '어제는 비가 많이 왔다.', 'labels': {'label': 0.8}}
    ]

# 데이터셋 구조 확인
print("\n데이터셋 샘플:")
for i in range(2):
    print(f"문장1: {sts_dataset[i]['sentence1']}")
    print(f"문장2: {sts_dataset[i]['sentence2']}")
    print(f"실제 유사도 점수: {sts_dataset[i]['labels']['label']:.1f}\n")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sts/train-00000-of-00001.parquet:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

sts/validation-00000-of-00001.parquet:   0%|          | 0.00/68.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]


데이터셋 샘플:
문장1: 무엇보다도 호스트분들이 너무 친절하셨습니다.
문장2: 무엇보다도, 호스트들은 매우 친절했습니다.
실제 유사도 점수: 4.9

문장1: 주요 관광지 모두 걸어서 이동가능합니다.
문장2: 위치는 피렌체 중심가까지 걸어서 이동 가능합니다.
실제 유사도 점수: 1.4



In [ ]:
# 문장을 벡터로 변환하는 함수 (Mean Pooling 적용)
def get_sentence_embedding(text, model, tokenizer):
    # 문장을 토큰화하고 텐서로 변환
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)

    # 모델을 통해 출력값 계산 (그래디언트 계산 비활성화로 속도 향상)
    with torch.no_grad():
        outputs = model(**inputs)

    # Mean Pooling: 토큰 임베딩의 평균을 계산하여 문장 전체의 벡터를 얻음
    # attention_mask를 사용하여 패딩 토큰은 제외하고 계산
    attention_mask = inputs['attention_mask']
    token_embeddings = outputs.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    return (sum_embeddings / sum_mask).cpu().numpy()[0]

# 코사인 유사도 계산 함수
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

In [ ]:
for model_name in model_names:
    print(f"--- {model_name} 모델 테스트 시작 ---")

    # 1. 해당 모델과 토크나이저 불러오기
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval() # 평가 모드로 설정

    model_predictions = [] # 모델이 예측한 유사도 점수
    true_labels = []       # 실제 유사도 점수

    # 2. 데이터셋의 모든 문장 쌍에 대해 반복
    for example in tqdm(sts_dataset, desc=f"{model_name} 처리 중"):
        sent1 = example['sentence1']
        sent2 = example['sentence2']
        true_label = example['labels']['label']

        # 3. 각 문장을 벡터로 임베딩
        embedding1 = get_sentence_embedding(sent1, model, tokenizer)
        embedding2 = get_sentence_embedding(sent2, model, tokenizer)

        # 4. 두 벡터의 코사인 유사도 계산
        pred_score = cosine_similarity(embedding1, embedding2)

        model_predictions.append(pred_score)
        true_labels.append(true_label)

    # 5. 스피어만 상관계수 계산
    # (사람이 매긴 점수와 모델이 예측한 점수의 순위 관계가 얼마나 유사한지 측정)
    spearman_corr, _ = spearmanr(true_labels, model_predictions)

    results[model_name] = spearman_corr * 100  # 보기 쉽게 100을 곱함

    print(f"--- {model_name} 모델 테스트 완료 ---")
    print(f"스피어만 상관계수: {results[model_name]:.2f}\n")

    # GPU 메모리 정리 (다음 모델을 위해)
    del model
    del tokenizer
    torch.cuda.empty_cache()

--- skt/kobert-base-v1 모델 테스트 시작 ---


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
print("========== 최종 성능 비교 결과 ==========")

# 점수가 높은 순으로 정렬
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)

for model_name, score in sorted_results:
    print(f"모델: {model_name.ljust(35)} | 스피어만 상관계수: {score:.2f}")

print("\n========== 분석 ==========")
best_model, best_score = sorted_results[0]
print(f"가장 높은 성능을 보인 모델은 '{best_model.split('/')[-1]}'이며, 상관계수 {best_score:.2f}를 기록했습니다.")
print("이 값은 모델이 예측한 문장 간의 의미 유사도 순위가 사람이 평가한 순위와 약 {:.0f}% 일치한다는 의미입니다.".format(best_score))
print("일반적으로 문장 임베딩에 특화된 훈련을 거친 'ko-sroberta-multitask' 모델이 가장 좋은 성능을 보일 가능성이 높습니다.")